In [ ]:
from array import array
import struct, os, random, math, pickle

In [ ]:
INPUT_PATH = './data'
INPUT_LAYER_SIZE = 28 * 28
HIDDEN_LAYER_SIZE = 16
OUTPUT_LAYER_SIZE = 10

In [ ]:
def load_and_preprocess_data(images_filepath, labels_filepath):
    # Read labels
    with open(labels_filepath, 'rb') as file:
        magic, size = struct.unpack(">II", file.read(8))
        if magic != 2049:
            raise ValueError(f'Magic number mismatch, expected 2049, got {magic}')
        labels = array("B", file.read())
    
    # Read images
    with open(images_filepath, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        if magic != 2051:
            raise ValueError(f'Magic number mismatch, expected 2051, got {magic}')
        image_data = array("B", file.read())
    
    # Process images
    images = []
    for i in range(size):
        start = i * rows * cols
        end = (i + 1) * rows * cols
        image = [
            [pixel / 255.0 for pixel in image_data[start + j * cols : start + (j + 1) * cols]]
            for j in range(rows)
        ]
        images.append([pixel for row in image for pixel in row])
    
    return images, labels

In [ ]:
training_images_filepath = os.path.join(INPUT_PATH, 'train-images')
training_labels_filepath = os.path.join(INPUT_PATH, 'train-labels')
test_images_filepath = os.path.join(INPUT_PATH, 'test-images')
test_labels_filepath = os.path.join(INPUT_PATH, 'test-labels')

TRAINING_IMAGES, TRAINING_LABELS = load_and_preprocess_data(training_images_filepath, training_labels_filepath)
TESTING_IMAGES, TESTING_LABELS = load_and_preprocess_data(test_images_filepath, test_labels_filepath)

In [ ]:
#logic functions
def relu(x):
    return max(0, x)

def relu_derivative(x):
    return 1 * (x > 0)

def softmax(x):
    max_x = max(x)
    exp_x = [math.exp(xi - max_x) for xi in x]
    sum_exp_x = sum(exp_x)
    softmax_x = [xi / sum_exp_x for xi in exp_x]
    return softmax_x

def compute_neuron_values(layer1_size, layer2_size, layer1, weights, biases):
    layer2 = []
    for i in range(layer2_size): # 16
        weighted_sum = 0
        for j in range(layer1_size): # 784
            weighted_sum += layer1[j] * weights[i][j]
        weighted_sum += biases[i]
        if layer2_size == OUTPUT_LAYER_SIZE:
            layer2.append(weighted_sum)
        else:
            layer2.append(relu(weighted_sum))
    return layer2

In [ ]:
def unpack():
    with open('nn/weights_biases.pkl', 'rb') as f:
        weights_biases_g = pickle.load(f)

    weights_input_to_hidden1 = weights_biases_g['weights_input_to_hidden1']
    biases_hidden1 = weights_biases_g['biases_hidden1']
    weights_hidden1_to_hidden2 = weights_biases_g['weights_hidden1_to_hidden2']
    biases_hidden2 = weights_biases_g['biases_hidden2']
    weights_hidden2_to_output = weights_biases_g['weights_hidden2_to_output']
    biases_output = weights_biases_g['biases_output']

    weights = [
       weights_input_to_hidden1, 
       weights_hidden1_to_hidden2, 
       weights_hidden2_to_output
    ]

    biases = [
        biases_hidden1, 
        biases_hidden2, 
        biases_output
    ]

    return weights, biases

In [ ]:
def forward_pass(input_layer, weights, biases):
    #unpack weights and biases as we need them for a forward pass

    weights_input_to_hidden1 = weights[0]
    weights_hidden1_to_hidden2 = weights[1]
    weights_hidden2_to_output = weights[2]

    biases_hidden1 = biases[0]
    biases_hidden2 = biases[1]
    biases_output = biases[2]

    #calculate activations for each layer

    hidden_layer_1 = compute_neuron_values(INPUT_LAYER_SIZE,
                                        HIDDEN_LAYER_SIZE,
                                        input_layer,
                                        weights_input_to_hidden1,
                                        biases_hidden1)


    hidden_layer_2 = compute_neuron_values(HIDDEN_LAYER_SIZE,
                                        HIDDEN_LAYER_SIZE,
                                        hidden_layer_1,
                                        weights_hidden1_to_hidden2,
                                        biases_hidden2)


    output_layer = compute_neuron_values(HIDDEN_LAYER_SIZE,
                                        OUTPUT_LAYER_SIZE,
                                        hidden_layer_2,
                                        weights_hidden2_to_output,
                                        biases_output)
    
    return [hidden_layer_1, hidden_layer_2, output_layer]

In [ ]:
def test_neural_network(test_images, test_labels, weights, biases):
    num_correct = 0
    num_test_images = len(test_images)
    
    for i in range(num_test_images):
        image = test_images[i]
        true_label = test_labels[i]
        
        output = forward_pass(image, weights, biases)[2]
        predicted_label = output.index(max(output))
        
        # Compare with true label
        if predicted_label == true_label:
            num_correct += 1
    
    # Calculate accuracy
    accuracy = num_correct / num_test_images * 100
    return accuracy

In [ ]:
def SGD(training_images_set, training_labels_set, weights, biases, learning_rate=0.001):
    #pure SGD: for every image, update weights and biases

    weights_input_to_hidden1 = weights[0]
    weights_hidden1_to_hidden2 = weights[1]
    weights_hidden2_to_output = weights[2]

    biases_hidden1 = biases[0]
    biases_hidden2 = biases[1]
    biases_output = biases[2]
    
    for i in range(len(training_images_set)):

        new_weights = [
            weights_input_to_hidden1, 
            weights_hidden1_to_hidden2, 
            weights_hidden2_to_output
        ]

        new_biases = [
            biases_hidden1, 
            biases_hidden2, 
            biases_output
        ]

        if i % 10000 == 0 and i != 0:
            print("accuracy at image ", i, ": ", test_neural_network(TESTING_IMAGES, TESTING_LABELS, new_weights, new_biases))
    
        input_layer = training_images_set[i]
        label = training_labels_set[i]
        neuron_activations = forward_pass(input_layer, new_weights, new_biases)

        hidden_layer_1 = neuron_activations[0]
        hidden_layer_2 = neuron_activations[1]
        output_layer = softmax(neuron_activations[2])
        
        actual_output = [0] * len(output_layer)
        actual_output[label] = 1

        #output deltas
        output_layer_deltas = [output_layer[i] - actual_output[i] for i in range(len(output_layer))]

        # layer 2 deltas ----------------
        hidden_layer_2_deltas = []
        for i in range(len(hidden_layer_2)): # len 16
            delta = 0
            for j in range(len(output_layer)): # len 10
                delta += weights_hidden2_to_output[j][i] * output_layer_deltas[j]
            delta *= relu_derivative(hidden_layer_2[i])
            hidden_layer_2_deltas.append(delta)
        

        #hidden layer 1 deltas ----------------
        hidden_layer_1_deltas = []

        for i in range(len(hidden_layer_1)):
            delta = 0
            for j in range(len(hidden_layer_2)):
                delta += weights_hidden1_to_hidden2[j][i] * hidden_layer_2_deltas[j]
            delta *= relu_derivative(hidden_layer_1[i])
            hidden_layer_1_deltas.append(delta)
        

        #now update the weights and biases
        for i in range(len(hidden_layer_2)): #len 16
            for j in range(len(output_layer)): # len 10
                weights_hidden2_to_output[j][i] -= learning_rate * output_layer_deltas[j] * hidden_layer_2[i]
            
        for i in range(len(hidden_layer_1)):
            for j in range(len(hidden_layer_2)):
                weights_hidden1_to_hidden2[j][i] -= learning_rate * hidden_layer_2_deltas[j] * hidden_layer_1[i]

        for i in range(len(input_layer)):
            for j in range(len(hidden_layer_1)):
                weights_input_to_hidden1[j][i] -= learning_rate * hidden_layer_1_deltas[j] * input_layer[i]

        for i in range(len(biases_output)):
            biases_output[i] -= learning_rate * output_layer_deltas[i]

        for i in range(len(biases_hidden2)):
            biases_hidden2[i] -= learning_rate * hidden_layer_2_deltas[i]

        for i in range(len(biases_hidden1)):
            biases_hidden1[i] -= learning_rate * hidden_layer_1_deltas[i]


    weights_biases = {
        'weights_input_to_hidden1': weights_input_to_hidden1,
        'biases_hidden1': biases_hidden1,
        'weights_hidden1_to_hidden2': weights_hidden1_to_hidden2,
        'biases_hidden2': biases_hidden2,
        'weights_hidden2_to_output': weights_hidden2_to_output,
        'biases_output': biases_output
    }

    with open('nn/weights_biases.pkl', 'wb') as f:
        pickle.dump(weights_biases, f)


    print("SGD Completed")

In [ ]:
def SGD_minibatch(training_images_set, training_labels_set, weights, biases, learning_rate=0.001, minibatch_size=32, batches=10000):
    # average gradients for minibatchand apply in backpass


    weights_input_to_hidden1 = weights[0]
    weights_hidden1_to_hidden2 = weights[1]
    weights_hidden2_to_output = weights[2]

    biases_hidden1 = biases[0]
    biases_hidden2 = biases[1]
    biases_output = biases[2]

    for batch in range(batches):
        minibatch_indices = random.sample(range(len(training_images_set)), minibatch_size)
        new_weights = [weights_input_to_hidden1, weights_hidden1_to_hidden2, weights_hidden2_to_output]
        new_biases = [biases_hidden1, biases_hidden2, biases_output]

        if batch % 1000 == 0 and batch != 0:
            print("Accuracy after", batch, "minibatches:", test_neural_network(TESTING_IMAGES, TESTING_LABELS, new_weights, new_biases))

        # Initialize accumulators for gradients and deltas
        final_output_deltas = [0] * OUTPUT_LAYER_SIZE
        final_hidden2_deltas = [0] * HIDDEN_LAYER_SIZE
        final_hidden1_deltas = [0] * HIDDEN_LAYER_SIZE

        gradients_hidden2_to_output = [[0 for _ in range(HIDDEN_LAYER_SIZE)] for _ in range(OUTPUT_LAYER_SIZE)]
        gradients_hidden1_to_hidden2 = [[0 for _ in range(HIDDEN_LAYER_SIZE)] for _ in range(HIDDEN_LAYER_SIZE)]
        gradients_input_to_hidden1 = [[0 for _ in range(INPUT_LAYER_SIZE)] for _ in range(HIDDEN_LAYER_SIZE)]

        for i in minibatch_indices:
            input_layer = training_images_set[i]
            label = training_labels_set[i]
            neuron_activations = forward_pass(input_layer, new_weights, new_biases)

            hidden_layer_1 = neuron_activations[0]
            hidden_layer_2 = neuron_activations[1]
            output_layer = softmax(neuron_activations[2])

            actual_output = [0] * len(output_layer)
            actual_output[label] = 1

            output_layer_deltas = [output_layer[i] - actual_output[i] for i in range(len(output_layer))]
            
            #hidden2 to output gradients
            for j in range(len(output_layer)):
                for i in range(len(hidden_layer_2)):
                    gradient = output_layer_deltas[j] * hidden_layer_2[i]
                    gradients_hidden2_to_output[j][i] += gradient


            #hidden1 to hidden2 output gradients
            hidden_layer_2_deltas = [0] * HIDDEN_LAYER_SIZE
            for j in range(len(hidden_layer_2)):
                delta = sum(weights_hidden2_to_output[k][j] * output_layer_deltas[k] for k in range(OUTPUT_LAYER_SIZE))
                delta *= relu_derivative(hidden_layer_2[j])
                hidden_layer_2_deltas[j] = delta

            for i in range(len(hidden_layer_1)):
                for j in range(len(hidden_layer_2)):
                    gradient = hidden_layer_2_deltas[j] * hidden_layer_1[i]
                    gradients_hidden1_to_hidden2[j][i] += gradient

            #input to hidden1 gradients
            hidden_layer_1_deltas = [0] * HIDDEN_LAYER_SIZE
            for j in range(len(hidden_layer_1)):
                delta = sum(weights_hidden1_to_hidden2[k][j] * hidden_layer_2_deltas[k] for k in range(HIDDEN_LAYER_SIZE))
                delta *= relu_derivative(hidden_layer_1[j])
                hidden_layer_1_deltas[j] = delta

            for i in range(len(input_layer)):
                for j in range(len(hidden_layer_1)):
                    gradient = hidden_layer_1_deltas[j] * input_layer[i]
                    gradients_input_to_hidden1[j][i] += gradient

            for i in range(OUTPUT_LAYER_SIZE):
                final_output_deltas[i] += output_layer_deltas[i]

            for i in range(HIDDEN_LAYER_SIZE):
                final_hidden2_deltas[i] += hidden_layer_2_deltas[i]

            for i in range(HIDDEN_LAYER_SIZE):
                final_hidden1_deltas[i] += hidden_layer_1_deltas[i]

        # Update weights and biases with averaged gradients

        for i in range(HIDDEN_LAYER_SIZE):
            for j in range(OUTPUT_LAYER_SIZE):
                weights_hidden2_to_output[j][i] -= learning_rate * gradients_hidden2_to_output[j][i] / minibatch_size

        for i in range(HIDDEN_LAYER_SIZE):
            for j in range(HIDDEN_LAYER_SIZE):
                weights_hidden1_to_hidden2[j][i] -= learning_rate * gradients_hidden1_to_hidden2[j][i] / minibatch_size

        for i in range(INPUT_LAYER_SIZE):
            for j in range(HIDDEN_LAYER_SIZE):
                weights_input_to_hidden1[j][i] -= learning_rate * gradients_input_to_hidden1[j][i] / minibatch_size

        for i in range(len(biases_output)):
            biases_output[i] -= learning_rate * final_output_deltas[i] / minibatch_size

        for i in range(len(biases_hidden2)):
            biases_hidden2[i] -= learning_rate * final_hidden2_deltas[i] / minibatch_size

        for i in range(len(biases_hidden1)):
            biases_hidden1[i] -= learning_rate * final_hidden1_deltas[i] / minibatch_size

    weights_biases = {
        'weights_input_to_hidden1': weights_input_to_hidden1,
        'biases_hidden1': biases_hidden1,
        'weights_hidden1_to_hidden2': weights_hidden1_to_hidden2,
        'biases_hidden2': biases_hidden2,
        'weights_hidden2_to_output': weights_hidden2_to_output,
        'biases_output': biases_output
    }

    with open('nn/weights_biases.pkl', 'wb') as f:
        pickle.dump(weights_biases, f)

    print("SGD minibatch Completed")

In [ ]:
def GD(training_images_set, training_labels_set, weights, biases, learning_rate=0.05, epochs=100):
    #SGD-mini but the minibatch is the entire dataset

    weights_input_to_hidden1 = weights[0]
    weights_hidden1_to_hidden2 = weights[1]
    weights_hidden2_to_output = weights[2]

    biases_hidden1 = biases[0]
    biases_hidden2 = biases[1]
    biases_output = biases[2]

    all_images = len(training_images_set)

    for epoch in range(epochs):
        new_weights = [weights_input_to_hidden1, weights_hidden1_to_hidden2, weights_hidden2_to_output]
        new_biases = [biases_hidden1, biases_hidden2, biases_output]

        if epoch % 10 == 0 and epoch != 0:
            print("Accuracy after", epoch, "GD epochs:", test_neural_network(TESTING_IMAGES, TESTING_LABELS, new_weights, new_biases))

        # Initialize accumulators for gradients and deltas
        final_output_deltas = [0] * OUTPUT_LAYER_SIZE
        final_hidden2_deltas = [0] * HIDDEN_LAYER_SIZE
        final_hidden1_deltas = [0] * HIDDEN_LAYER_SIZE

        gradients_hidden2_to_output = [[0 for _ in range(HIDDEN_LAYER_SIZE)] for _ in range(OUTPUT_LAYER_SIZE)]
        gradients_hidden1_to_hidden2 = [[0 for _ in range(HIDDEN_LAYER_SIZE)] for _ in range(HIDDEN_LAYER_SIZE)]
        gradients_input_to_hidden1 = [[0 for _ in range(INPUT_LAYER_SIZE)] for _ in range(HIDDEN_LAYER_SIZE)]

        for i in range(all_images):
            input_layer = training_images_set[i]
            label = training_labels_set[i]
            neuron_activations = forward_pass(input_layer, new_weights, new_biases)

            hidden_layer_1 = neuron_activations[0]
            hidden_layer_2 = neuron_activations[1]
            output_layer = softmax(neuron_activations[2])

            actual_output = [0] * len(output_layer)
            actual_output[label] = 1

            output_layer_deltas = [output_layer[i] - actual_output[i] for i in range(len(output_layer))]
            
            #hidden2 to output gradients
            for j in range(len(output_layer)):
                for i in range(len(hidden_layer_2)):
                    gradient = output_layer_deltas[j] * hidden_layer_2[i]
                    gradients_hidden2_to_output[j][i] += gradient


            #hidden1 to hidden2 output gradients
            hidden_layer_2_deltas = [0] * HIDDEN_LAYER_SIZE
            for j in range(len(hidden_layer_2)):
                delta = sum(weights_hidden2_to_output[k][j] * output_layer_deltas[k] for k in range(OUTPUT_LAYER_SIZE))
                delta *= relu_derivative(hidden_layer_2[j])
                hidden_layer_2_deltas[j] = delta

            for i in range(len(hidden_layer_1)):
                for j in range(len(hidden_layer_2)):
                    gradient = hidden_layer_2_deltas[j] * hidden_layer_1[i]
                    gradients_hidden1_to_hidden2[j][i] += gradient

            #input to hidden1 gradients
            hidden_layer_1_deltas = [0] * HIDDEN_LAYER_SIZE
            for j in range(len(hidden_layer_1)):
                delta = sum(weights_hidden1_to_hidden2[k][j] * hidden_layer_2_deltas[k] for k in range(HIDDEN_LAYER_SIZE))
                delta *= relu_derivative(hidden_layer_1[j])
                hidden_layer_1_deltas[j] = delta

            for i in range(len(input_layer)):
                for j in range(len(hidden_layer_1)):
                    gradient = hidden_layer_1_deltas[j] * input_layer[i]
                    gradients_input_to_hidden1[j][i] += gradient

            for i in range(OUTPUT_LAYER_SIZE):
                final_output_deltas[i] += output_layer_deltas[i]

            for i in range(HIDDEN_LAYER_SIZE):
                final_hidden2_deltas[i] += hidden_layer_2_deltas[i]

            for i in range(HIDDEN_LAYER_SIZE):
                final_hidden1_deltas[i] += hidden_layer_1_deltas[i]

        # Update weights and biases with averaged gradients

        for i in range(HIDDEN_LAYER_SIZE):
            for j in range(OUTPUT_LAYER_SIZE):
                weights_hidden2_to_output[j][i] -= learning_rate * gradients_hidden2_to_output[j][i] / all_images

        for i in range(HIDDEN_LAYER_SIZE):
            for j in range(HIDDEN_LAYER_SIZE):
                weights_hidden1_to_hidden2[j][i] -= learning_rate * gradients_hidden1_to_hidden2[j][i] / all_images

        for i in range(INPUT_LAYER_SIZE):
            for j in range(HIDDEN_LAYER_SIZE):
                weights_input_to_hidden1[j][i] -= learning_rate * gradients_input_to_hidden1[j][i] / all_images
                
        for i in range(len(biases_output)):
            biases_output[i] -= learning_rate * final_output_deltas[i] / all_images

        for i in range(len(biases_hidden2)):
            biases_hidden2[i] -= learning_rate * final_hidden2_deltas[i] / all_images

        for i in range(len(biases_hidden1)):
            biases_hidden1[i] -= learning_rate * final_hidden1_deltas[i] / all_images

    weights_biases = {
        'weights_input_to_hidden1': weights_input_to_hidden1,
        'biases_hidden1': biases_hidden1,
        'weights_hidden1_to_hidden2': weights_hidden1_to_hidden2,
        'biases_hidden2': biases_hidden2,
        'weights_hidden2_to_output': weights_hidden2_to_output,
        'biases_output': biases_output
    }

    with open('nn/weights_biases.pkl', 'wb') as f:
        pickle.dump(weights_biases, f)

    print("GD Completed")


In [15]:
def he_initialization(fan_in, fan_out):
    std = math.sqrt(2.0 / fan_in)
    return [[random.gauss(0, std) for _ in range(fan_in)] for _ in range(fan_out)]

#to get a fresh set of weights and biases
weights_biases = {
    'weights_input_to_hidden1': he_initialization(INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE),
    'biases_hidden1': [0.0 for _ in range(HIDDEN_LAYER_SIZE)],
    'weights_hidden1_to_hidden2': he_initialization(HIDDEN_LAYER_SIZE, HIDDEN_LAYER_SIZE),
    'biases_hidden2': [0.0 for _ in range(HIDDEN_LAYER_SIZE)],
    'weights_hidden2_to_output': he_initialization(HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE),
    'biases_output': [0.0 for _ in range(OUTPUT_LAYER_SIZE)]
}

#now that we have initialised them, save it with pickle
with open('nn/weights_biases.pkl', 'wb') as f:
    pickle.dump(weights_biases, f)

weights, biases = unpack()

In [ ]:
# Pick the optimisation method
SGD(TRAINING_IMAGES, TRAINING_LABELS, weights, biases)
#SGD_minibatch(TRAINING_IMAGES, TRAINING_LABELS, weights, biases)
#GD(TRAINING_IMAGES, TRAINING_LABELS, weights, biases)

In [ ]:
weights, biases = unpack()
test_neural_network(TESTING_IMAGES, TESTING_LABELS, weights, biases)